
# Krylov Subspace

The pure iteration has the form

$$Px_{k+1} = (P-A)x_k+b$$

For simplicity, we choose $P = I$ and $x_1 = b$, thus the whole iterations are only to do with $A$ and $b$

$$\begin{array}{ll}
x_1 = b\\
x_2 = (I-A)b+b = 2b-Ab\\
x_3 = (I-A)x_1 + b = 3b-3Ab + A^2b
\end{array}$$

The point is simple but important: $x_j$ is a combination of $b, Ab, \cdots, A^{j-1}b$. And the linear combinations of these vectors form the $j^{th}$ Krylov subspace.

$$\begin{array}{ll}
\text{Krylov matrix} &K_j = [b\ Ab\ A^2b\ \cdots\ A^{j-1}b]\\
\text{Krylov subspace} &\mathcal{K}_j = \text{all combinations of }b,Ab,\cdots, A^{j-1}b
\end{array}$$

The problem is what is the best choise of $x_j$ in the Krylov subspace. Here are four different approaches to choosing a good $x_j$ in $\mathcal{K}_j$.

1. The residual $r_j = b-Ax_j$ is orthogonal to $\mathcal{K}_j$ (**Conjugate Gradients**).
2. The residual $r_j$ has minimum norm for $x_j$ in $\mathcal{K}_j$ (**GMRES and MINRES**).
3. $r_j$ is orthogonal to a different space $\mathcal{K}_j(A^T)$ (**BiConjugate Gradients**).
4. The error $e_j$ has minimum norm (**SYMMLQ**).



---------------
# Arnoldi's Method


Before discussing the Conjugate Gradient method, we should extract orthonormal basis from the Krylov matrix to represent the basis of the Krylov subspace for simplifying the upcoming calculation.

$$\text{Krylov subspace}\quad \mathcal{K}_j = \text{all combinations of }q_1,q_2,\cdots, q_{j-1}$$

Each new $q_j$ comes from orthogonalizing $t=Aq_{j-1}$ (which has the same direction as $A^{j-1}b$) to the basis vectors $q_1,\cdots, q_{j-1}$ that are already chosen. The iteration to compute these orthonormal $q$'s is **Arnoldi's method**. This method is essentially the Gram-Schmidt idea.
 

| Step |  Pseudocode | Discription |
| ---- | :---------- | :----------- |
|<img width=30/>|<img width=200/>|<img width=200/>|
| 0   | $q_1 = b/\|b\|$            | Normalize $b$ to $\|q_1\| = 1$ |
|&nbsp;| for $j = 1,\cdots,n-1$       | Start computation of $q_{j+1}$|
| 1   | &emsp; $t = Aq_j\qquad\ $      | one matrix multiplication|
|&nbsp;| &emsp; for $i = 1,\cdots,j$     | $t$ is in the space $\mathcal{K}_{j+1}$| 
| 2   | &emsp; &emsp; $h_{ij} = q_i^Tt$   | $h_{ij}q_i =$ projection of $t$ on $q_i$|
| 3   | &emsp; &emsp; $t = t-h_{ij}q_i$   | Subtract that projection |
|&nbsp;| &emsp; end                | $t$ is orthogonal to $q_1, \cdots, q_j$ |
| 4   | &emsp; $h_{j+1,j} = \|t\|$      | Compute the length of $t$ |
| 5   | &emsp; $q_{j+1} = t/h_{j+1,j}$  | Normalize $t$ to $\|q_{j+1}\|=1$ |
|&nbsp;| end                    | $q_1,\cdots,q_n$ are orthonormal |

<font color='#aaaaaa'>*Custom table alignment seems to be supported after version 5.8 in Jupyter notebook. <a href='https://github.com/jupyter/notebook/pull/4130'>#4130</a>*</font>

where $Aq_j$ is in the $\mathcal{K}_{j+1}$ subspace, that is to say, $Aq_j$ is the linear combination of $q_1, \cdots, q_j, q_{j+1}$, and the weights of these orthonormal vectors are $h_{1j},\cdots, h_{jj}, h_{j+1,j}$. The Arnoldi's method can therefore be denoted by

$$AQ = \begin{bmatrix}
\\
\\
Aq_1 &Aq_2 &\cdots &Aq_n\\
\\
\\
\end{bmatrix}
=
\begin{bmatrix}
\\
\\
q_1 &q_2 &\cdots &q_n\\
\\
\\
\end{bmatrix}
\begin{bmatrix}
h_{1,1} &h_{1,2} &\cdots &h_{1,n-1} &h_{1,n}\\
h_{2,1} &h_{2,2} &\cdots &h_{2,n-1} &h_{2,n}\\
0    &h_{3,2} &\cdots &h_{3,n-1} &h_{3,n}\\
\vdots &\vdots &\ddots &\vdots &\vdots\\
0 &0 &\cdots &h_{n, n-1} &h_{n,n}
\end{bmatrix}
=QH$$

This matrix $H$ is upper triangular plus one lower diagonal, which makes it "upper Hessenberg".


# Lanczos method

If $A$ is symmetric, then

$$\left.\begin{array}{ll}
AQ=QH\Rightarrow A=QHQ^T\\
A^T=A
\end{array}\right\}\Rightarrow H=H^T \Rightarrow H=\begin{bmatrix}
h_{1,1} &h_{1,2} &\cdots &0 &0\\
h_{2,1} &h_{2,2} &\cdots &0 &0\\
0    &h_{3,2} &\cdots &0 &0\\
\vdots &\vdots &\ddots &\vdots &\vdots\\
0 &0 &\cdots &h_{n, n-1} &h_{n,n}
\end{bmatrix}$$

$H$ is tridiagonal. Since $H$ has three nonzeros in its rows and columns. So computing $q_{j+1}$ only involves $q_j$ and $q_{j-1}$:

$$Aq_j = h_{j+1,j}q_{j+1} + h_{j,j}q_j + h_{j-1,j}q_{j-1}$$

This is the **Lanczos iteration**.